# 🚀 Advanced Python Concepts: Decorators, Context Managers, and Metaclasses

**Welcome!** This notebook takes a deep dive into some of Python's more advanced and powerful features: Decorators, Context Managers, and Metaclasses. These concepts allow for sophisticated metaprogramming, elegant resource management, and code reuse patterns, often underpinning major frameworks and libraries. Understanding them unlocks a deeper appreciation of Python's flexibility and power.

**Target Audience:** Intermediate to advanced Python developers seeking to understand metaprogramming techniques and advanced language features.

**Learning Objectives:**
*   Master the creation and application of decorators for modifying function and class behavior.
*   Understand and implement the context management protocol (`with` statement) using classes and `contextlib`.
*   Grasp the concept of metaclasses, how classes are created, and how to customize this process.
*   Apply these concepts to practical scenarios like logging, timing, resource management, API registration, and code validation.
*   Learn best practices, common pitfalls, and performance considerations for each concept.
*   Prepare for advanced Python interview questions related to these topics.

## 1. Decorators: Enhancing Functions and Methods

**Concept:** Decorators provide a way to modify or enhance functions or methods in a clean, readable, and reusable manner. They are a form of metaprogramming and leverage Python's first-class function capabilities.

**Syntactic Sugar:** The `@decorator_name` syntax above a function definition is syntactic sugar for applying the decorator function to the defined function.

```python
# The @ syntax:
@my_decorator
def say_hello():
    print("Hello!")

# Is equivalent to:
def say_hello():
    print("Hello!")
say_hello = my_decorator(say_hello) # Decorator function wraps the original
```

**Analogy: Function Wrapping Paper**
Think of a decorator as special wrapping paper for a function. The original function (the gift) remains inside, but the wrapping paper (`@decorator`) adds extra features or behavior (like adding a bow, a security tag, or a 'fragile' label) before the gift is presented (the function is called).

**Core Idea:** Decorators are higher-order functions – they take a function as input and typically return a *new* function (often a nested `wrapper` function) that includes the original function's logic plus the added behavior.

### 1.1 Simple Function Decorator

In [ ]:
import functools
import logging
import time
from typing import Callable, Any

logging.basicConfig(level=logging.INFO, format='%(levelname)s:%(name)s:%(message)s', force=True)
logger = logging.getLogger('DecoratorDemo')

def log_calls(func: Callable) -> Callable:
    """Decorator that logs when a function is called and its arguments."""
    # Use functools.wraps to preserve original function metadata (name, docstring, etc.)
    @functools.wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        # Log before calling the original function
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]
        signature = ", ".join(args_repr + kwargs_repr)
        logger.info(f"Calling {func.__name__}({signature})")
        
        # Call the original function
        result = func(*args, **kwargs)
        
        # Log after calling (optional)
        logger.info(f"{func.__name__} returned {result!r}")
        return result
    return wrapper

# Apply the decorator
@log_calls
def greet(name: str, enthusiasm: int = 1) -> str:
    """Returns a greeting message."""
    message = f"Hello, {name}" + "!" * enthusiasm
    return message

@log_calls
def calculate_sum(a: int, b: int, c: int = 0) -> int:
    """Calculates the sum of numbers."""
    return a + b + c

# Call the decorated functions
print("--- Testing @log_calls --- ")
greeting = greet("Alice", enthusiasm=3)
total = calculate_sum(10, 20, c=5)

print(f"\nGreeting result: {greeting}")
print(f"Sum result: {total}")

# Check preserved metadata thanks to @functools.wraps
print(f"\nGreet function name: {greet.__name__}")
print(f"Greet function docstring: {greet.__doc__}")

### 1.2 Decorators with Arguments

To create a decorator that accepts arguments, you need an extra layer of nesting: a factory function that takes the arguments and *returns* the actual decorator function.

In [ ]:
import functools
import time
from typing import Callable, Any

def repeat(num_times: int) -> Callable:
    """Decorator factory: Creates a decorator to run a function 'num_times'."""
    if not isinstance(num_times, int) or num_times <= 0:
        raise ValueError("Number of times must be a positive integer.")
        
    # This is the actual decorator
    def decorator_repeat(func: Callable) -> Callable:
        @functools.wraps(func)
        def wrapper_repeat(*args: Any, **kwargs: Any) -> None:
            print(f"Running {func.__name__} {num_times} times:")
            for i in range(num_times):
                print(f"  Run {i+1}: ", end="")
                result = func(*args, **kwargs)
                # In this case, we might not care about the return value
                # or maybe return a list of results
            print("Finished repeating.")
            # Return value of the last call, or None, or list of results?
            # Design decision - let's return None for simplicity here.
        return wrapper_repeat
    return decorator_repeat

# Using the decorator factory
@repeat(num_times=3)
def say_whee():
    """Prints whee!"""
    print("Whee!")

@repeat(1)
def print_hello(name):
    print(f"Hello {name}")

print("--- Testing @repeat decorator --- ")
say_whee()
print("---")
print_hello("Bob")

# Example of error handling in factory
# try:
#     @repeat(num_times=-1)
#     def should_fail(): pass
# except ValueError as e:
#     print(f"\nCaught expected error: {e}")

### 1.3 Class Decorators

While less common than function decorators for simple tasks, you can also use classes as decorators. The class needs to implement `__call__` or `__init__` appropriately.

*   **`__init__` approach:** The `__init__` method receives the decorated function/class. You typically store it and implement `__call__` to provide the wrapped behavior.
*   **`__call__` approach:** The decorator class `__init__` might take arguments (like the factory function). The `__call__` method then receives the function to be decorated and returns the wrapper.

In [ ]:
import functools
from typing import Any, Callable

# Using __init__ to receive the function
class CallCounter:
    def __init__(self, func: Callable):
        functools.update_wrapper(self, func) # Preserve metadata
        self.func = func
        self.num_calls = 0
        print(f"Initializing CallCounter for {func.__name__}")

    def __call__(self, *args: Any, **kwargs: Any) -> Any:
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__}")
        return self.func(*args, **kwargs)

@CallCounter
def example_func_counted(x):
    """An example function to be counted."""
    print(f"  Executing example_func_counted with {x}")
    return x * 2

print("--- Testing Class Decorator (CallCounter) --- ")
result1 = example_func_counted(10)
result2 = example_func_counted(20)
result3 = example_func_counted(30)
print(f"Final call count for {example_func_counted.__name__}: {example_func_counted.num_calls}")

# Using __call__ as the decorator factory (less common)
# class DecoratorWithArgs:
#     def __init__(self, arg1, arg2):
#         print(f"Initializing decorator with args: {arg1}, {arg2}")
#         self.arg1 = arg1
#         self.arg2 = arg2
# 
#     def __call__(self, func: Callable) -> Callable:
#         print(f"Applying decorator to {func.__name__}")
#         @functools.wraps(func)
#         def wrapper(*args, **kwargs):
#             print(f"Wrapper called with decorator args: {self.arg1}, {self.arg2}")
#             return func(*args, **kwargs)
#         return wrapper
# 
# @DecoratorWithArgs(arg1="value1", arg2=123)
# def another_func():
#     print("Executing another_func")
# 
# print("\n--- Testing Class Decorator Factory ---")
# another_func()

### 1.4 Decorating Classes

Decorators can also be applied to classes. The decorator function receives the class object itself and should typically return a (potentially modified) class object.

In [ ]:
from typing import Type, Any
import inspect

def add_repr(cls: Type) -> Type:
    """Decorator to automatically add a basic __repr__ to a class."""
    print(f"Decorating class {cls.__name__} to add __repr__")
    
    def custom_repr(self) -> str:
        # Get attributes using __dict__ or inspect
        # Be careful with properties or complex state
        attrs = []
        # Simple version using __dict__ (won't show properties etc.)
        # for k, v in self.__dict__.items():
        #     attrs.append(f"{k}={v!r}")
        
        # More robust using inspect (still might need care)
        try:
            sig = inspect.signature(cls.__init__) # Assumes attributes match init params
            for param_name in sig.parameters:
                if param_name == 'self': continue
                try:
                    value = getattr(self, param_name)
                    attrs.append(f"{param_name}={value!r}")
                except AttributeError:
                    pass # Attribute might not exist if init logic is complex
        except ValueError: # Handle classes without standard __init__ (like builtins)
             attrs = [f"{k}={v!r}" for k, v in self.__dict__.items() if not k.startswith('_')]
             
        return f"{cls.__name__}({', '.join(attrs)})"

    # Add the new __repr__ method to the class
    # Only add if __repr__ doesn't already exist or is the default object.__repr__
    if '__repr__' not in cls.__dict__ or cls.__repr__ is object.__repr__:
        setattr(cls, '__repr__', custom_repr)
        
    return cls

@add_repr
class Point:
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y

    # No __repr__ defined here

@add_repr
class Config:
    def __init__(self, setting1: str, setting2: bool = True):
        self.setting1 = setting1
        self._internal = 123 # Should not appear in default repr
        # setting2 is not directly stored if logic changes it
        self.is_enabled = setting2 
        
    # Already has a __repr__ - decorator should ideally not overwrite
    # def __repr__(self): 
    #    return f"Config(setting1='{self.setting1}')" 

print("--- Testing Class Decorator --- ")
p = Point(10, 20)
c = Config("valueA", setting2=False)

# The __repr__ added by the decorator is used
print(f"Point repr: {p!r}")
print(f"Config repr: {c!r}") # Repr might vary based on implementation

### 1.5 Use Cases & Best Practices

**Common Use Cases:**
*   **Logging:** Recording function calls, arguments, return values, execution time.
*   **Timing:** Measuring function execution time.
*   **Caching/Memoization:** `@functools.lru_cache`, `@functools.cache`.
*   **Access Control/Authorization:** Checking user permissions before executing a function.
*   **Input Validation/Transformation:** Ensuring arguments meet criteria or converting them before passing to the function.
*   **Registration:** Registering functions or classes in a central registry (e.g., plugin systems, web framework route handlers).
*   **Adding Attributes:** Class decorators can add methods or attributes to classes.

**Best Practices:**
*   **Use `@functools.wraps`:** Always preserve the original function's metadata.
*   **Keep Decorators Focused:** Each decorator should ideally do one specific thing.
*   **Clear Naming:** Name decorators descriptively.
*   **Consider Performance:** Decorators add a layer of function call overhead. Usually negligible, but be mindful in performance-critical loops.
*   **Debugging:** Debugging decorated functions can sometimes be tricky. Understand that the debugger might step into the wrapper first.
*   **Order Matters:** If multiple decorators are applied, they are executed from bottom-up (the one closest to the `def` runs first, its result is passed to the one above, etc.).

## 2. Context Managers: The `with` Statement

**Concept:** Context managers provide a structured way to manage resources (like files, network connections, locks) by ensuring that setup and teardown operations are always performed, even if errors occur during the resource's use.

**The `with` Statement:** This is the primary way to use context managers.
```python
with setup_resource() as resource_variable:
    # Use the resource_variable here
    # ... code that might raise exceptions ...

# --- After the 'with' block --- 
# The resource is guaranteed to be cleaned up (e.g., file closed, lock released)
```

**Analogy: Automatic Door Closer**
Think of a sensitive room (e.g., a darkroom, a cleanroom). A context manager is like a special door mechanism. 
1.  `__enter__`: You swipe your card (`setup_resource()`), the door opens, and maybe the lights turn on (setup).
2.  `with ... as resource:`: You enter the room and do your work.
3.  `__exit__`: When you leave the `with` block (either by finishing normally or by running out screaming because of an error!), the door mechanism *automatically* closes the door, turns off the lights, etc. (cleanup), regardless of what happened inside.

**Context Management Protocol:** An object must implement `__enter__()` and `__exit__()` to be used as a context manager.
*   `__enter__(self)`: Called when entering the `with` block. Its return value is assigned to the variable after `as` (if present). Used for setup.
*   `__exit__(self, exc_type, exc_value, traceback)`: Called when exiting the `with` block. 
    *   Receives exception details if an exception occurred within the block (otherwise `None`).
    *   Performs cleanup actions.
    *   Can optionally suppress the exception by returning `True` (otherwise the exception propagates).

### 2.1 Built-in Example: `open()`

In [ ]:
# File objects returned by open() are context managers.
file_path = "context_demo.txt"

try:
    with open(file_path, "w", encoding='utf-8') as f:
        print(f"Inside 'with': File '{f.name}' open? {!f.closed}")
        f.write("Managed by context manager!\n")
        # Simulate an error
        # result = 1 / 0 
    # --- Exiting the 'with' block --- 
    print(f"Outside 'with': File open? {f.closed}") # File is automatically closed

except ZeroDivisionError:
    print("Caught error inside 'with'.")
    # Check file status even after error
    # Note: 'f' might not be defined here if open() failed, 
    # hence why checking f.closed outside the 'except' is safer 
    # IF you need to access f after an error. 
    # But the key point is __exit__ was still called to close.
    # print(f"Outside 'with' after error: File open? {f.closed}") 
except IOError as e:
    print(f"Error opening/writing file: {e}")

# Verify file is closed even if we exited scope where f was defined
# We need to re-open to check content or status reliably here
try:
    with open(file_path, "r", encoding='utf-8') as f_check:
         print(f"Content after 'with': {f_check.read().strip()}")
except Exception as e:
     print(f"Error reopening file: {e}")

### 2.2 Creating Class-Based Context Managers

In [ ]:
import time
from typing import Optional, Type

class TimerContext:
    """A context manager to time a block of code."""
    def __init__(self, description: str = "Block"):
        self.description = description
        self.start_time: Optional[float] = None
        self.end_time: Optional[float] = None
        self.elapsed: Optional[float] = None

    def __enter__(self) -> 'TimerContext':
        """Called when entering the 'with' block. Records start time."""
        print(f"\nEntering timer context for: {self.description}")
        self.start_time = time.perf_counter()
        return self # Return self to allow access via 'as' variable

    def __exit__(self, 
                 exc_type: Optional[Type[BaseException]], 
                 exc_value: Optional[BaseException], 
                 traceback: Optional[Any]) -> Optional[bool]:
        """Called when exiting the 'with' block. Records end time and calculates duration."""
        self.end_time = time.perf_counter()
        if self.start_time is not None: # Ensure start_time was set
            self.elapsed = self.end_time - self.start_time
            status = "completed successfully" if exc_type is None else f"exited with {exc_type.__name__}"
            print(f"Exiting timer context for: {self.description} ({status})")
            print(f"Elapsed time: {self.elapsed:.6f} seconds")
        else:
            print(f"Exiting timer context for: {self.description} (start time not recorded)")
        
        # Return False (or None) by default to propagate any exceptions
        # Return True to suppress the exception (use carefully!)
        return False 

# --- Using the class-based context manager --- 
print("--- Testing Class-Based TimerContext --- ")
with TimerContext("Sleep Operation") as timer1:
    print("  Inside 'with': Sleeping for 0.2 seconds...")
    time.sleep(0.2)
    print(f"  Inside 'with': Accessing timer object elapsed: {timer1.elapsed}") # Still None

# Access elapsed time after the block exits
print(f"After 'with': Timer 1 elapsed: {timer1.elapsed:.6f} seconds")

# Example with an error
try:
    with TimerContext("Failing Operation") as timer2:
        print("  Inside 'with': Performing operation...")
        result = 10 / 0 # This will raise ZeroDivisionError
except ZeroDivisionError:
    print("Caught ZeroDivisionError outside the 'with' block (propagated).")
    # Check the timer object after the exception
    if timer2 and timer2.elapsed is not None:
       print(f"After 'with' (error): Timer 2 elapsed before error: {timer2.elapsed:.6f} seconds")

### 2.3 Creating Function-Based Context Managers (`contextlib.contextmanager`)

**Modern Practice:** The `contextlib` module provides utilities for creating context managers more concisely, often using generator functions decorated with `@contextmanager`.

**How it works:**
1.  Define a generator function decorated with `@contextlib.contextmanager`.
2.  The code *before* the `yield` statement acts as the `__enter__` part.
3.  The value yielded is what's assigned to the `as` variable (if any).
4.  The code *after* the `yield` statement acts as the `__exit__` part (cleanup). Exception handling around the `yield` is necessary to mimic `__exit__`'s exception handling.

In [ ]:
import contextlib
import time
from typing import Generator, Any

@contextlib.contextmanager
def simple_timer(description: str = "Block") -> Generator[None, None, None]:
    """Function-based context manager using contextlib for timing."""
    start_time = None
    print(f"\nEntering function timer context for: {description}")
    start_time = time.perf_counter()
    
    try:
        # yield control back to the 'with' block
        # Can yield a value here to be used with 'as'
        yield 
        # --- Code after yield runs upon exiting the 'with' block --- 
        end_time = time.perf_counter()
        elapsed = end_time - start_time
        print(f"Exiting function timer context for: {description} (completed successfully)")
        print(f"Elapsed time: {elapsed:.6f} seconds")
    except Exception as e:
        # --- If an exception occurred in the 'with' block --- 
        end_time = time.perf_counter()
        # Ensure start_time was set before calculating elapsed
        elapsed = (end_time - start_time) if start_time is not None else 0 
        print(f"Exiting function timer context for: {description} (exited with {type(e).__name__})")
        print(f"Elapsed time before error: {elapsed:.6f} seconds")
        # Exception is automatically re-raised unless caught and suppressed here
        raise # Re-raise the exception to mimic default __exit__ behavior
    # finally:
        # Code here runs regardless, similar to finally in __exit__
        # print("Inside contextmanager's finally")

# --- Using the function-based context manager --- 
print("--- Testing Function-Based simple_timer --- ")
with simple_timer("Task A"):
    print("  Doing Task A...")
    time.sleep(0.15)

try:
    with simple_timer("Task B (Failing)"):
        print("  Doing Task B...")
        time.sleep(0.05)
        raise ValueError("Something went wrong in Task B")
except ValueError as e:
    print(f"Caught expected error outside 'with': {e}")

### 2.4 Use Cases & Best Practices

**Common Use Cases:**
*   **Resource Management:** Files (`open`), network connections (sockets, DB connections), locks (`threading.Lock`).
*   **Setup/Teardown:** Temporarily changing state (e.g., `os.chdir` in `tempfile.TemporaryDirectory`, setting global configurations).
*   **Transaction Management:** Ensuring database commits or rollbacks.
*   **Timing/Profiling:** Measuring execution time of code blocks.

**Best Practices:**
*   **Use `with`:** Always use the `with` statement for objects that support the context management protocol.
*   **Prefer `contextlib.contextmanager`:** For simpler context managers, the decorator approach is often more concise than writing a full class.
*   **Proper Exception Handling in `__exit__` / `contextmanager`:** Decide whether to suppress exceptions (return `True` from `__exit__` or handle within the `except` block of `@contextmanager`) or let them propagate (return `False`/`None` or re-raise).
*   **Idempotency:** Ensure `__enter__` and `__exit__` can handle being called multiple times if relevant (though `with` statement usually prevents this).
*   **Keep `__enter__` / `__exit__` Focused:** They should focus solely on resource setup and teardown, not complex application logic.

## 3. Metaclasses: The Magic of Class Creation

**Concept:** Metaclasses are the 'classes of classes'. Just as a class defines how *instances* are created and behave, a metaclass defines how *classes themselves* are created and behave. This is arguably the most advanced and least commonly needed feature among these three.

**How Classes Are Created:**
When you define a class `class MyClass(BaseClass): ...`, Python performs roughly these steps:
1.  The body of `MyClass` is executed, creating its attributes (methods, variables) in a temporary namespace.
2.  Python determines the appropriate **metaclass**. By default, this is `type`. If `BaseClass` has a different metaclass, or if `metaclass=MyMeta` is specified in the class definition, that metaclass is used.
3.  The metaclass (e.g., `type`) is called with the class name, base classes, and the class attributes dictionary: `MyClass = type('MyClass', (BaseClass,), {attributes...})`.
4.  The metaclass's `__new__` method is called to create the class object.
5.  The metaclass's `__init__` method is called to initialize the newly created class object.

**Analogy: The Blueprint Factory**
*   **Objects** are like houses built from a blueprint.
*   **Classes** are like the blueprints used to build the houses.
*   **Metaclasses** are like the factory or the architectural firm that *designs and creates the blueprints* themselves. They can enforce rules about how blueprints must be designed (e.g., all house blueprints must specify the number of bathrooms) or automatically add features to every blueprint they produce (e.g., adding a standard disclaimer).

**Key Takeaway:** Metaclasses intercept class creation, allowing you to modify or inspect the class *before* it's finalized.

### 3.1 Understanding `type`

`type` is the default metaclass. It can also be used directly to create classes dynamically.

In [ ]:
# Standard class definition
class MyRegularClass:
    x = 10
    def greet(self):
        print("Hello from regular class")

instance = MyRegularClass()
print(f"Instance type: {type(instance)}")
print(f"Class type (its metaclass): {type(MyRegularClass)}")

# Creating the same class dynamically using type()
# type(name, bases, dct)
def greet_method(self):
    print("Hello from dynamic class")

MyDynamicClass = type(
    'MyDynamicClass', # Class name
    (object,),        # Base classes (tuple)
    {                 # Attributes dictionary
        'x': 20,
        'greet': greet_method
    }
)

dynamic_instance = MyDynamicClass()
print(f"\nDynamic Instance type: {type(dynamic_instance)}")
print(f"Dynamic Class type (its metaclass): {type(MyDynamicClass)}")
print(f"Dynamic instance attribute x: {dynamic_instance.x}")
dynamic_instance.greet()

### 3.2 Creating a Simple Metaclass

Metaclasses typically inherit from `type` and override `__new__` and/or `__init__`.
*   `mcs.__new__(mcs, name, bases, attrs)`: Creates the *class* object. `mcs` is the metaclass itself. Should return the new class object (usually by calling `super().__new__(...)`). Use this to modify `name`, `bases`, or `attrs` *before* the class is created.
*   `mcs.__init__(cls, name, bases, attrs)`: Initializes the newly created *class* object `cls`. Use this to perform actions *after* the class is created (e.g., register the class, add computed attributes).

In [ ]:
from typing import Dict, Tuple, Type

# --- Simple Metaclass using __new__ --- 
class EnsureAttributesMeta(type):
    """Metaclass to ensure created classes have specific attributes."""
    def __new__(mcs, name: str, bases: Tuple[Type, ...], attrs: Dict[str, Any]) -> Type:
        print(f"\n[Meta] Creating class '{name}' using {mcs.__name__}")
        print(f"[Meta] Bases: {bases}")
        print(f"[Meta] Attributes found: {list(attrs.keys())}")
        
        # Enforce that specific attributes must exist
        required_attrs = ['description', 'version']
        for attr_name in required_attrs:
            if attr_name not in attrs:
                raise TypeError(f"Class '{name}' must define the attribute '{attr_name}'")
                
        # Add a class attribute automatically
        attrs['__metameta__'] = 'Ensured by EnsureAttributesMeta'
        
        # Call the parent's __new__ to actually create the class object
        new_class = super().__new__(mcs, name, bases, attrs)
        print(f"[Meta] Finished creating class '{name}'")
        return new_class
    
    # Optional: __init__ for post-creation initialization of the class
    # def __init__(cls, name, bases, attrs):
    #     print(f"[Meta Init] Initializing class {name}")
    #     super().__init__(name, bases, attrs)

# --- Use the metaclass --- 
print("--- Defining classes with EnsureAttributesMeta ---")

class MyPlugin(metaclass=EnsureAttributesMeta):
    description = "A sample plugin."
    version = "1.0"
    
    def run(self):
        print(f"Running {self.description}")

print(f"MyPlugin created. Metameta attribute: {MyPlugin.__metameta__}")
plugin_instance = MyPlugin()
plugin_instance.run()

# This definition would fail because 'version' is missing
try:
    class IncompletePlugin(metaclass=EnsureAttributesMeta):
        description = "An incomplete plugin."
        # version is missing
        def execute(self):
             pass
except TypeError as e:
    print(f"\nCaught expected error defining IncompletePlugin: {e}")

### 3.3 Use Cases & Alternatives

**Common Use Cases (Often in Frameworks/Libraries):**
*   **ORMs (Object-Relational Mappers):** Metaclasses inspect class attributes (like `Column`, `String`) to automatically generate database mapping code, SQL queries, etc. (e.g., SQLAlchemy, Django ORM).
*   **API Registration:** Automatically registering API endpoints defined as methods within specific classes.
*   **Plugin Architectures:** Discovering and registering plugin classes automatically.
*   **Automatic Method/Attribute Generation:** Adding boilerplate methods or attributes to classes based on their definition.
*   **Class Validation:** Enforcing constraints on class definitions (like required methods or specific attribute types).

**Metaclass Warning:**
> "Metaclasses are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don’t (the people who actually need them know with certainty that they need them, and don’t need an explanation about why)." — Tim Peters

**Alternatives (Often Simpler):**

1.  **Class Decorators:** As seen earlier, class decorators can modify a class *after* it's created. They are often sufficient for tasks like adding methods or registering the class, and are generally easier to understand.
2.  **`__init_subclass__` (Python 3.6+):** A special class method automatically called when a subclass is defined. Useful for customizing subclass creation without needing a full metaclass (e.g., registering subclasses).
3.  **Inheritance and Mixins:** Standard object-oriented techniques for sharing code and behavior.
4.  **Descriptor Protocol:** For controlling attribute access (`__get__`, `__set__`, `__delete__`). Used by `@property`.

In [ ]:
# --- Alternative: Using __init_subclass__ --- 
plugin_registry = {}

class BasePlugin:
    # This method is called automatically when a subclass of BasePlugin is defined
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        plugin_name = cls.__name__.lower().replace('plugin', '')
        print(f"[init_subclass] Registering plugin: {plugin_name} -> {cls.__name__}")
        if plugin_name in plugin_registry:
            raise TypeError(f"Plugin name '{plugin_name}' already registered.")
        plugin_registry[plugin_name] = cls

# Subclasses automatically get registered via __init_subclass__
class FilePlugin(BasePlugin):
    def load(self, path):
        print(f"FilePlugin loading {path}")

class NetworkPlugin(BasePlugin):
    def connect(self, url):
        print(f"NetworkPlugin connecting to {url}")

# This would fail due to duplicate name 'file'
# class SecondFilePlugin(BasePlugin): 
#     pass

print("\n--- Plugin Registry (using __init_subclass__) ---")
print(plugin_registry)

# Instantiate a plugin from the registry
if 'network' in plugin_registry:
    NetworkPluginClass = plugin_registry['network']
    network_plugin = NetworkPluginClass()
    network_plugin.connect("http://example.com")

### 3.4 Best Practices & Pitfalls

**Best Practices:**
*   **Use Only When Necessary:** Consider alternatives (class decorators, `__init_subclass__`) first.
*   **Keep Them Simple:** Metaclass logic should be clear and focused on class creation/modification.
*   **Inherit from `type`:** Custom metaclasses should usually inherit from `type`.
*   **Use `super()`:** Properly call `super().__new__(...)` and `super().__init__(...)` within your metaclass methods.
*   **Clear Documentation:** Document why a metaclass is needed and what it does.

**Pitfalls:**
*   **Over-Complexity:** Easily leads to code that is hard to understand, debug, and maintain.
*   **Metaclass Conflicts:** If a class inherits from multiple base classes that have *different* metaclasses (and neither is a subclass of the other), Python raises a `TypeError` because it doesn't know which metaclass to use.
*   **Debugging:** Debugging metaclass code itself can be challenging as it runs during the import/class definition phase.
*   **Action at a Distance:** Metaclasses can modify classes in non-obvious ways, making it harder to trace behavior.

## 4. Combined Challenge: Auto-Logged Resource

**Goal:** Create a class that acts as a context manager, where entry and exit points (and any errors) are automatically logged using a decorator applied via a metaclass.

**Tasks:**

1.  **Logging Decorator:** Create a function decorator `@log_resource_actions` that logs entry (`__enter__`), exit (successful or with error), and the result/exception for any method it decorates.
2.  **Metaclass (`LogContextMeta`):**
    *   Create a metaclass that automatically applies the `@log_resource_actions` decorator to the `__enter__` and `__exit__` methods of any class created with this metaclass *if* those methods exist.
    *   Hint: You'll need to inspect the class attributes (`attrs`) in `__new__` or `__init__`, find the methods, wrap them with the decorator, and update the attributes dictionary *before* creating the class with `super().__new__`.
3.  **Base Context Class:** Create a simple base class `ManagedResource` (or similar) that uses `LogContextMeta` as its metaclass. It doesn't need to implement `__enter__` or `__exit__` itself.
4.  **Concrete Resource Class:** Create a class `DatabaseConnection` that inherits from `ManagedResource` and implements:
    *   `__init__(self, db_name: str)`
    *   `__enter__(self)`: Simulates opening a connection (e.g., prints a message) and returns `self`.
    *   `__exit__(self, exc_type, exc_value, traceback)`: Simulates closing the connection (e.g., prints a message).
5.  **Test:** Use the `DatabaseConnection` class within a `with` statement. Observe the automatic logging provided by the metaclass applying the decorator.
   Test both a successful `with` block and one that raises an exception inside.

**(Bonus):** Modify the metaclass or decorator to allow passing a custom logger name when applying the metaclass.

In [ ]:
# --- Solution Space for Challenge ---
import functools
import logging
from typing import Callable, Any, Type, Tuple, Dict, Optional

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(name)s - %(message)s', force=True)

# 1. Logging Decorator
def log_resource_actions(func: Callable) -> Callable:
    @functools.wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        logger = logging.getLogger(func.__qualname__) # Log using function's qualified name
        instance = args[0] if args else None # Assume first arg is 'self'
        context_info = f" (Instance: {instance!r})" if instance else ""
        
        result = None
        exc_info = None
        try:
            # Special handling for __exit__ args
            if func.__name__ == '__exit__':
                exc_type, exc_value, tb = args[1], args[2], args[3]
                if exc_type:
                     logger.info(f"Entering {func.__name__}{context_info} with Exception: {exc_type.__name__}")
                     exc_info = (exc_type, exc_value, tb)
                else:
                     logger.info(f"Entering {func.__name__}{context_info} (Normal Exit)")
            else:
                 logger.info(f"Entering {func.__name__}{context_info}")
            
            result = func(*args, **kwargs)
            
            if func.__name__ != '__exit__': # Don't log exit result if normal exit
                 logger.info(f"Exiting {func.__name__}{context_info} successfully, returned: {result!r}")
            elif not exc_info: # Log successful exit from __exit__
                 logger.info(f"Exiting {func.__name__}{context_info} successfully.")
                 
            return result
        except Exception as e:
            logger.exception(f"Exception raised in {func.__name__}{context_info}: {e}")
            if func.__name__ == '__exit__':
                # Avoid logging exception twice if it happened during __exit__ itself
                # but we need to make sure it propagates if __exit__ doesn't suppress
                pass 
            raise # Re-raise exception
        finally:
            # This part might be redundant if logging is done before return/raise
            # logger.info(f"Finally exiting {func.__name__}{context_info}")
            pass
    return wrapper

# 2. Metaclass
class LogContextMeta(type):
    def __new__(mcs, name: str, bases: Tuple[Type, ...], attrs: Dict[str, Any]) -> Type:
        logger = logging.getLogger(f"{mcs.__name__}.{name}")
        logger.info(f"Creating class '{name}' with LogContextMeta")
        
        # Decorate __enter__ and __exit__ if they exist
        if '__enter__' in attrs and callable(attrs['__enter__']):
            logger.debug(f"Applying @log_resource_actions to {name}.__enter__")
            attrs['__enter__'] = log_resource_actions(attrs['__enter__'])
            
        if '__exit__' in attrs and callable(attrs['__exit__']):
            logger.debug(f"Applying @log_resource_actions to {name}.__exit__")
            attrs['__exit__'] = log_resource_actions(attrs['__exit__'])
            
        # Create the class using the (potentially modified) attributes
        new_class = super().__new__(mcs, name, bases, attrs)
        return new_class

# 3. Base Context Class (using the metaclass)
class ManagedResource(metaclass=LogContextMeta):
    """Base class for resources managed by LogContextMeta."""
    pass

# 4. Concrete Resource Class
class DatabaseConnection(ManagedResource): # Inherits metaclass
    def __init__(self, db_name: str):
        self.db_name = db_name
        self._connection_state = "closed"
        logging.info(f"DatabaseConnection '{self.db_name}' initialized.")

    def __enter__(self) -> 'DatabaseConnection':
        # Logging here is now handled by the decorator! 
        # print(f"[Original Enter] Opening connection to {self.db_name}...")
        self._connection_state = "open"
        return self

    def __exit__(self, 
                 exc_type: Optional[Type[BaseException]], 
                 exc_value: Optional[BaseException], 
                 traceback: Optional[Any]) -> Optional[bool]:
        # Logging here is now handled by the decorator! 
        # print(f"[Original Exit] Closing connection to {self.db_name}. Error: {exc_type}")
        self._connection_state = "closed"
        # Let's not suppress exceptions
        return False 
        
    def query(self, sql: str):
        if self._connection_state != "open":
            raise ConnectionError("Database connection is not open.")
        print(f"  Executing query on '{self.db_name}': {sql}")
        return [('result1',), ('result2',)]
        
    def __repr__(self):
        return f"<DatabaseConnection(db_name='{self.db_name}', state='{self._connection_state}')>"

# 5. Test
print("\n--- Testing Auto-Logged Context Manager ---")
print("\n--- Scenario 1: Successful Operation ---")
try:
    with DatabaseConnection("prod_db") as db_conn:
        print("  Inside 'with': Connection object:", db_conn)
        results = db_conn.query("SELECT * FROM users")
        print(f"  Inside 'with': Query results: {results}")
    print("Successfully completed Scenario 1.")
except Exception as e:
    print(f"Scenario 1 failed unexpectedly: {e}")

print("\n--- Scenario 2: Operation with Exception ---")
try:
    with DatabaseConnection("staging_db") as db_conn_err:
        print("  Inside 'with': Connection object:", db_conn_err)
        db_conn_err.query("UPDATE products SET price = price / 0") # Will cause ZeroDivisionError
    print("Scenario 2 completed (Should not happen).")
except ZeroDivisionError:
    print("Caught expected ZeroDivisionError outside 'with' block (Scenario 2).")
except Exception as e:
    print(f"Scenario 2 failed unexpectedly: {e}")


## 5. Conclusion

Decorators, Context Managers, and Metaclasses represent powerful, albeit progressively more complex, features of Python. 

*   **Decorators** provide elegant syntax for wrapping functions and methods, ideal for cross-cutting concerns like logging, timing, and access control.
*   **Context Managers** (`with` statement) ensure robust resource management and cleanup, crucial for handling files, locks, and connections.
*   **Metaclasses** offer deep control over class creation itself, enabling sophisticated framework development and code generation patterns, but should be used judiciously due to their complexity.

Mastering decorators and context managers is highly beneficial for writing clean, reusable, and reliable Python code. Metaclasses, while less frequently needed, offer insight into Python's object model and provide solutions for advanced library and framework design challenges. Always consider the simplest effective solution first before reaching for the most powerful tool.